In [1]:
# source activate bigdata

In [2]:
import sys
import os
import time
import pandas as pd
import json
import shutil
from functools import singledispatch

In [3]:
print(os.environ['CONDA_DEFAULT_ENV'])

bigdata


In [4]:
from platform import python_version

print(python_version())

3.7.6


In [5]:
from cryptography.fernet import Fernet 
import os 
  
cred_filename = 'CredFile.ini'
key_file = 'key.key'
  
key = '' 
  
with open('.key.key','r') as key_in: 
    key = key_in.read().encode() 
  
# If you want the Cred file to be of one  
# time use uncomment the below line 
# s.remove(key_file) 
  
f = Fernet(key) 
with open(cred_filename,'r') as cred_in: 
    lines = cred_in.readlines() 
    config = {} 
    for line in lines: 
        tuples = line.rstrip('\n').split('=',1) 
        if tuples[0] in ('Username ','Password '): 
            config[tuples[0]] = tuples[1] 
  
    usr_id = config['Username ']
    passwrd = f.decrypt(config['Password '].encode()).decode()

In [6]:
sys.path.append('../lib/pysmb-1.1.28/python3/')
sys.path.append('../lib/iptcinfo3/')
# del sys.path[-1]
# sys.path

In [7]:
from smb.SMBConnection import SMBConnection
import tempfile
from iptcinfo3 import IPTCInfo

In [8]:
TODAY = pd.Timestamp.today().date()
TODAY_FILE = TODAY.strftime('%Y-%m-%d')

In [9]:
import configparser

smbconfig = configparser.ConfigParser()
smbconfig.read('smb.ini')

path = smbconfig['shares']['Path']
top = smbconfig['shares']['TopLvl']
client_sys = smbconfig['shares']['Client']
server_sys = smbconfig['shares']['Server']
server_ip = smbconfig['shares']['ServerIP']
share = smbconfig['shares']['Share']

['smb.ini']

In [10]:
try:
    conn = SMBConnection(usr_id, passwrd, client_sys, server_sys, domain='',
                     use_ntlm_v2=True, sign_options=2, is_direct_tcp=False)
    connected = conn.connect(server_ip, 139)
except:
    print('Zugriff auf SMB-share nicht möglich!')

In [11]:
Response = conn.listShares(timeout=30)  # obtain a list of shares
print('Shares auf: ' + server_sys)

for i in range(len(Response)):  # iterate through the list of shares
    print("  Share[",i,"] =", Response[i].name)
    try:
        # list the files on each share
        Response2 = conn.listPath(Response[i].name,'/',timeout=30)
        print('    Files on: ' + server_sys + '/' + "  Share[",i,"] =", Response[i].name)
        for i in range(len(Response2)):
            print("    File[",i,"] =", Response2[i].filename)
    except:
        print('### can not access the resource')

Shares auf: centosvm01.intern.generica.net
  Share[ 0 ] = print$
### can not access the resource
  Share[ 1 ] = users
    Files on: centosvm01.intern.generica.net/  Share[ 1 ] = users
    File[ 0 ] = .
    File[ 1 ] = ..
    File[ 2 ] = ThecusNAS
    File[ 3 ] = Alfresco-Export
    File[ 4 ] = 2019-07-18_Copy_ThecusNAS.txt
    File[ 5 ] = 2019-07-18_Copy_AlfExport.txt
    File[ 6 ] = Daten von LokalDaten
  Share[ 2 ] = IPC$
### can not access the resource


In [12]:
iptc_info_keys = ['object name', 'edit status', 'editorial update', 'urgency', 'subject reference',
                  'category', 'supplemental category', 'fixture identifier', 'keywords',
                  'content location code', 'content location name', 'release date', 'release time',
                  'expiration date', 'expiration time', 'special instructions', 'action advised',
                  'reference service', 'reference date', 'reference number', 'date created', 'time created',
                  'digital creation date', 'digital creation time', 'originating program', 'program version',
                  'object cycle', 'by-line', 'by-line title', 'city', 'sub-location', 'province/state',
                  'country/primary location code', 'country/primary location name',
                  'original transmission reference', 'headline', 'credit', 'source', 'copyright notice',
                  'contact', 'caption/abstract', 'local caption', 'writer/editor', 'image type',
                  'image orientation', 'language identifier', 'custom1', 'custom2', 'custom3', 'custom4',
                  'custom5', 'custom6', 'custom7', 'custom8', 'custom9', 'custom10', 'custom11', 'custom12',
                  'custom13', 'custom14', 'custom15', 'custom16', 'custom17', 'custom18', 'custom19', 'custom20']

In [13]:
log_file = open(TODAY_FILE + '_log.txt', 'w', encoding='utf8')

In [14]:
# single-dispatch generic function
@singledispatch
def decd(data):
    return data.decode()

@decd.register(str)
def _(data):
    return data

In [15]:
def smbwalk(conn, share, top = u'/'):
    
    dirs, nondirs, file_ext, file_count = [], [], set(), 0
    
    if not isinstance(conn, SMBConnection):
        raise TypeError("SMBConnection required")
        
    names = conn.listPath(share, top)
    
    breaker = 0
    
    for name in names:
        if name.isDirectory:
            if name.filename not in [u'.', u'..']:
                dirs.append(name.filename)
        else:
            filename, file_extension = os.path.splitext(name.filename)
            path = top + '/' + name.filename
            abs_path = 'smb://' + server_sys + '/' + share + '/' + top + '/'
            log_file.write( 'Datei: ' + path + ', Extension: ' + file_extension + '\n' )
            # -------------- .ARW-part --------------
            if any(file_extension in s for s in ['.ARW']):
                scale_factor = 1200
                for suff_ix in ['_bg', '_md', '_sm']:
                    temp_arw_file = tempfile.NamedTemporaryFile(suffix=".arw")
                    temp_jpg_file = tempfile.NamedTemporaryFile(suffix=".jpg")
                    file_attributes, filesize = conn.retrieveFile(share, path, temp_arw_file, timeout=60)
                    #
                    # os.system("ffmpeg -i \"{0}\" -y -vf scale={1}:-1 {2}.jpg".format(
                    #    temp_arw_file.name, scale_factor, filename + suff_ix))
                    #
                    # os.system("convert \"arw:{0}\" -layers merge -resize {1} -define dng:use-camera-wb=true \
                    #          \"./tmp/{2}.jpg\"".format(temp_arw_file.name, scale_factor, filename + suff_ix))
                    #
                    os.system("darktable-cli \"{0}\" \"{1}\" --width={2} --verbose".format(
                        temp_arw_file.name, temp_jpg_file, scale_factor))
                    scale_factor = scale_factor - 500
                    print("Moving {0} to {1}".format(temp_jpg_file.name, abs_path + filename + suff_ix + '.jpg' ))
                    shutil.move(os.path.abspath(temp_jpg_file.name), abs_path + filename + suff_ix + '.jpg')
            # -------------- .jpg-part --------------
            if any(file_extension in s for s in ['.jpg', '.JPG']):
                file_obj = tempfile.NamedTemporaryFile()
                file_attributes, filesize = conn.retrieveFile(share, path, file_obj, timeout=60)
                # https://github.com/jamesacampbell/iptcinfo3
                file_info = IPTCInfo(file_obj)
                for key in iptc_info_keys:
                    if file_info[key] is not None:
                        if isinstance(file_info[key], list):
                            for num, elem in enumerate(file_info[key], start=1):
                                log_file.write( 'IPTC \'' + key + '(' + str(num) + ')\': ' +
                                               decd(elem) + '\n')
                        else:
                            log_file.write( 'IPTC \'' + key + '\': ' + decd(file_info[key]) + '\n' )
            log_file.write('Letzter Zugriff: ' + time.ctime(name.last_access_time) + '\n' )
            log_file.write('Letzte Änderung: ' + time.ctime(name.last_write_time) + '\n' )
            log_file.write('Letzte Attributsänderung: ' + time.ctime(name.last_attr_change_time) + '\n' )
            log_file.write('Dateigröße: ' + str(round(name.file_size/(1024*1024), 2)) + 'MB' + '\n')
            # log_file.write(os.popen("identify -verbose \"{0}\"".format(
            #        abs_path + name.filename)).read() + '\n')
            nondirs.append(name)
            file_ext.add(file_extension)
            file_count += 1
            
    # yielding top level directory incl. content
    # yield top
    # yield dirs
    yield nondirs
    yield file_ext
    yield file_count
    
    for name in dirs:
        new_path = os.path.join(top, name)
        for directory in smbwalk(conn, share, new_path):
            # yielding subdirectory inkl. content
            yield directory

In [16]:
log_file = open(TODAY_FILE + '_log.txt', 'w', encoding='utf8')

In [17]:
files_count = 0
file_extensions = set()


for elem in smbwalk(conn, share, top):
    if isinstance(elem, list):
        for x in elem:
            filename, delimiter, file_extension = x.filename.rpartition('.')
            # ToDo: do something with files in dir-list

    if isinstance(elem, set):
        file_extensions.update(elem)
    if isinstance(elem, int):
        files_count = files_count + elem

Moving /tmp/tmppsn212ul.jpg to smb://centosvm01.intern.generica.net/users/ThecusNAS/InHouse Produktfotos/sortiert/DSC00001_bg.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'smb://centosvm01.intern.generica.net/users/ThecusNAS/InHouse Produktfotos/sortiert/DSC00001_bg.jpg'

In [ ]:
log_summary = open(TODAY_FILE + '_summary.txt', 'w+')

In [ ]:
log_summary.write(str(file_extensions) + '\n')
log_summary.write(str(files_count) + '\n')

In [ ]:
log_file.close()
log_summary.close()
conn.close()